# 머신러닝 맛보기  
- 일자: 2020/08/23
- 작성자: 김보겸
- 참고자료: 파이썬으로 데이터 주무르기(민형기), Intro to DS SPRINGRUSH by 박하람  

안녕하세요 여러분! 이번에는 머신러닝에 대해서 진짜 사아알짝 맛만 보는 시간을 가져볼까합니다. 교안을 보다보면 벡터도 나오고, 선형대수도 나오고 해서 멘붕이 오시겠지만...😇 그걸 전부 다 알 필요는 없습니다! 알고리즘 모델을 만들 때 수학이 필요한건데, 저희는 만들어진 모델을 가져다 쓰기만 할거거든요. 그냥 이런게 있구나~ 정도만 알면 됩니다 😚

## Naive Bayes Classifier의 이해 - 영문  
머신러닝에는 지도학습과 비지도학습이 있는데요, 이 나이브 베이즈 분류기는 지도학습에 속합니다. 컴퓨터를 학습시킬 때 데이터에 대한 레이블이 있기 때문에(정답이 있다!) '지도'(supervised)학습이라 부릅니다. 

더 자세히 알고 싶다면 아래 링크를 참고해주세요!  
[머신러닝 종류](https://www.playsw.or.kr/artificial/view/playswtv/672?currentTab=ai_data_video&path=artificial)

머신러닝은 기본적으로 어떠한 알고리즘을 만들기 위해 학습시킬 train data와 이 train data를 통해 학습한 알고리즘으로 새롭게 예측해볼 test data를 요구합니다. 여기에서도 train을 통해 알고리즘을 학습시키고 이를 새로운 test data를 넣어 예측해볼 겁니다.

In [1]:
# 자연어처리에 필요한 모듈들을 깔아줍시다
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-13.0.2"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-s

In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# 연습용 데이터 train 
train = [('i like you','pos'),
         ('i hate you', 'neg'),
         ('you like me', 'neg'),
         ('i like her',' pos')]

train 변수 안에 있는 모든 단어들을 가져오는 코드를 작성하려고 하는데요. 그에 앞서서 중복되는 단어들을 제거하려고 해요. 저번 시간처럼 set을 활용해봅시다

In [4]:
for sentence in train: 
  for word in word_tokenize(sentence[0]): 
    print(word)

i
like
you
i
hate
you
you
like
me
i
like
her


In [5]:
all_words = set(word.lower() for sentence in train 
                for word in word_tokenize(sentence[0]))
all_words

{'hate', 'her', 'i', 'like', 'me', 'you'}

자~ 이제 모든 단어가 있는 말뭉치(all_words)가 만들어졌어요. 그럼 이 말뭉치를 기준으로 train의 각 문장에 어떤 단어가 있고 어떤 단어가 없는지 검사해보겠습니다

In [6]:
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1])
          for x in train]
t

[({'hate': False,
   'her': False,
   'i': True,
   'like': True,
   'me': False,
   'you': True},
  'pos'),
 ({'hate': True,
   'her': False,
   'i': True,
   'like': False,
   'me': False,
   'you': True},
  'neg'),
 ({'hate': False,
   'her': False,
   'i': False,
   'like': True,
   'me': True,
   'you': True},
  'neg'),
 ({'hate': False,
   'her': True,
   'i': True,
   'like': True,
   'me': False,
   'you': False},
  ' pos')]

결과를 해석해보면 train의 첫번째 문장에는 hate가 없고, i는 있다는 뜻이에요. 코드가 조금 복잡해서 이해하기 어려울 수 있지만, 대충 무슨 의미인지 파악하기만 됩니다. 자세한건 이후 머신러닝 커리에서 배우면 돼요~ 이것을 활용해 나이브 베이즈 분류기를 동작시키겠습니다.

In [7]:
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()

Most Informative Features
                      me = False             pos : neg    =      1.5 : 1.0
                    hate = False             pos : neg    =      1.5 : 1.0
                       i = True              pos : neg    =      1.5 : 1.0
                    like = True              pos : neg    =      1.5 : 1.0
                     her = False             neg : pos    =      1.1 : 1.0
                     you = True              neg : pos    =      1.1 : 1.0


첫번째 문장 hate = False의 의미는 hate가 없을 때, 긍정일 확률이 1.5, 부정일 확률이 1이라는 의미입니다. train 데이터로 학습시켰으니 test 데이터(새로운 데이터)도 이에 기반해서 잘 예측할 수 있는지 확인해보겠습니다.

In [9]:
test_sentence = 'i like MeRui'
# 위와 같이 말뭉치를 기준으로 어떤 단어가 있고, 또 어떤 단어는 없는지 검사해봅시다
test_sent_features = {word.lower(): 
                      (word in word_tokenize(test_sentence.lower()))
                      for word in all_words}

test_sent_features

{'hate': False,
 'her': False,
 'i': True,
 'like': True,
 'me': False,
 'you': False}

In [10]:
# 위 결과를 classify에 넣으면
classifier.classify(test_sent_features)

'pos'

test_sentence의 문장은 긍정을 나타내는 문장이라 잘 판단했네요. 분류기를 가동시킬 문장이 많으면 신뢰성있는 결론을 얻을 것이라 하는데요, 아주 간단하지만 잘 동작한다는 점에서 '나이브' 베이즈 분류기라고 할 수 있습니다.

## Naive Byes Classifier의 이해 - 한글

한글로도 하려면 형태소 분석을 거쳐야 해요. 형태소 분석을 거치지 않을 경우 어떻게 되는지부터 살펴봅시다.

In [11]:
from konlpy.tag import Okt
pos_tagger = Okt()

# 연습 데이터
train = [('메리가 좋아', 'pos'), 
         ('고양이도 좋아', 'pos'),
         ('난 수업이 지루해', 'neg'),
         ('메리는 이쁜 고양이야', 'pos'),
         ('난 마치고 메리랑 놀거야', 'pos')]

In [12]:
# train 데이터의 말뭉치 생성
all_words = set(word.lower() for sentence in train
                        for word in word_tokenize(sentence[0]))
all_words

{'고양이도',
 '고양이야',
 '난',
 '놀거야',
 '마치고',
 '메리가',
 '메리는',
 '메리랑',
 '수업이',
 '이쁜',
 '좋아',
 '지루해'}

형태소 분석을 하지 않으면 '고양이도'와 '고양이야'가 / '메리가', '메리는', '메리랑'이 각각 다르게 뽑히게 됩니다. 문장을 제대로 분석하려면 명사와 조사로 나뉘어서 뽑혀야 하는데 붙어서 나와버려요. 우선 이 상태에서 분석을 계속 진행해보겠습니다 

In [13]:
# 말뭉치를 기준으로 단어의 유무 검사
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1])
                                                        for x in train]
t

[({'고양이도': False,
   '고양이야': False,
   '난': False,
   '놀거야': False,
   '마치고': False,
   '메리가': True,
   '메리는': False,
   '메리랑': False,
   '수업이': False,
   '이쁜': False,
   '좋아': True,
   '지루해': False},
  'pos'),
 ({'고양이도': True,
   '고양이야': False,
   '난': False,
   '놀거야': False,
   '마치고': False,
   '메리가': False,
   '메리는': False,
   '메리랑': False,
   '수업이': False,
   '이쁜': False,
   '좋아': True,
   '지루해': False},
  'pos'),
 ({'고양이도': False,
   '고양이야': False,
   '난': True,
   '놀거야': False,
   '마치고': False,
   '메리가': False,
   '메리는': False,
   '메리랑': False,
   '수업이': True,
   '이쁜': False,
   '좋아': False,
   '지루해': True},
  'neg'),
 ({'고양이도': False,
   '고양이야': True,
   '난': False,
   '놀거야': False,
   '마치고': False,
   '메리가': False,
   '메리는': True,
   '메리랑': False,
   '수업이': False,
   '이쁜': True,
   '좋아': False,
   '지루해': False},
  'pos'),
 ({'고양이도': False,
   '고양이야': False,
   '난': True,
   '놀거야': True,
   '마치고': True,
   '메리가': False,
   '메리는': False,
   '메리랑': True,
   '수업이': False,
   '이쁜': 

In [15]:
# classifier에 검사내용을 넣어 일종의 예측 가이드라인을 만듭니다
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()

Most Informative Features
                       난 = True              neg : pos    =      2.5 : 1.0
                      좋아 = False             neg : pos    =      1.5 : 1.0
                    고양이도 = False             neg : pos    =      1.1 : 1.0
                      이쁜 = False             neg : pos    =      1.1 : 1.0
                     메리는 = False             neg : pos    =      1.1 : 1.0
                     메리가 = False             neg : pos    =      1.1 : 1.0
                    고양이야 = False             neg : pos    =      1.1 : 1.0
                     마치고 = False             neg : pos    =      1.1 : 1.0
                     메리랑 = False             neg : pos    =      1.1 : 1.0
                     놀거야 = False             neg : pos    =      1.1 : 1.0


train 내용을 바탕으로 test 데이터를 넣어 분석해봅시다

In [16]:
test_sentence = '난 수업이 마치면 메리랑 놀거야'
test_sent_features = {word.lower(): 
                      (word in word_tokenize(test_sentence.lower()))
                      for word in all_words}
test_sent_features

{'고양이도': False,
 '고양이야': False,
 '난': True,
 '놀거야': True,
 '마치고': False,
 '메리가': False,
 '메리는': False,
 '메리랑': True,
 '수업이': True,
 '이쁜': False,
 '좋아': False,
 '지루해': False}

In [17]:
classifier.classify(test_sent_features)

'neg'

이런! '난 수업이 마치면 메리랑 놀거야'라는 문장을 부정적인 문장이라고 분석했네요. 자, 이제 형태소 분석의 중요성을 잘 아셨을겁니다

In [18]:
def tokenize(doc): 
  return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

konlpy를 만든 Lucy Park에 따르면 위의 함수처럼 태그를 붙여주는 것이 유리하다고 해요. tokenize 함수를 사용해서 train 문장을 형태소 분석해보겠습니다.

tokenize가 뭔지 잘 모르겠다면, 저번 교안에서 제가 알려드린 내용을 다시 읽어보시면 됩니다!👨‍🏫
- 문자열 단위를 토큰(token)이라고 하고 이렇게 문자열을 토큰으로 나누는 작업을 토큰 생성(tokenizing)이라고 하고
- 문자열을 토큰으로 분리하는 함수를 토큰 생성 함수(tokenizer)라고 합니다. 

In [19]:
train = [('메리가 좋아', 'pos'), 
         ('고양이도 좋아', 'pos'),
         ('난 수업이 지루해', 'neg'),
         ('메리는 이쁜 고양이야', 'pos'),
         ('난 마치고 메리랑 놀거야', 'pos')] 

train_docs = [(tokenize(row[0]), row[1]) for row in train]
train_docs

[(['메리/Noun', '가/Josa', '좋다/Adjective'], 'pos'),
 (['고양이/Noun', '도/Josa', '좋다/Adjective'], 'pos'),
 (['난/Noun', '수업/Noun', '이/Josa', '지루하다/Adjective'], 'neg'),
 (['메리/Noun', '는/Josa', '이쁘다/Adjective', '고양이/Noun', '야/Josa'], 'pos'),
 (['난/Noun', '마치/Noun', '고/Josa', '메리/Noun', '랑/Josa', '놀다/Verb'], 'pos')]

In [20]:
# 이 코드를 아래 코드에서는 한 줄로 적어준 겁니다!
for d in train_docs: 
  for t in d[0]: 
    print(t)

메리/Noun
가/Josa
좋다/Adjective
고양이/Noun
도/Josa
좋다/Adjective
난/Noun
수업/Noun
이/Josa
지루하다/Adjective
메리/Noun
는/Josa
이쁘다/Adjective
고양이/Noun
야/Josa
난/Noun
마치/Noun
고/Josa
메리/Noun
랑/Josa
놀다/Verb


In [21]:
tokens = [t for d in train_docs for t in d[0]]
tokens

['메리/Noun',
 '가/Josa',
 '좋다/Adjective',
 '고양이/Noun',
 '도/Josa',
 '좋다/Adjective',
 '난/Noun',
 '수업/Noun',
 '이/Josa',
 '지루하다/Adjective',
 '메리/Noun',
 '는/Josa',
 '이쁘다/Adjective',
 '고양이/Noun',
 '야/Josa',
 '난/Noun',
 '마치/Noun',
 '고/Josa',
 '메리/Noun',
 '랑/Josa',
 '놀다/Verb']

이렇게 전체 말뭉치(tokens)를 만들었습니다. 아래 함수는 해당 문장에 있는 형태소가 tokens 안에 들어있는지 확인하는 것이에요

In [22]:
def term_exists(doc): 
  return {word: (word in set(doc)) for word in tokens}

In [23]:
train_xy = [(term_exists(d), c) for d,c in train_docs]
train_xy

[({'가/Josa': True,
   '고/Josa': False,
   '고양이/Noun': False,
   '난/Noun': False,
   '놀다/Verb': False,
   '는/Josa': False,
   '도/Josa': False,
   '랑/Josa': False,
   '마치/Noun': False,
   '메리/Noun': True,
   '수업/Noun': False,
   '야/Josa': False,
   '이/Josa': False,
   '이쁘다/Adjective': False,
   '좋다/Adjective': True,
   '지루하다/Adjective': False},
  'pos'),
 ({'가/Josa': False,
   '고/Josa': False,
   '고양이/Noun': True,
   '난/Noun': False,
   '놀다/Verb': False,
   '는/Josa': False,
   '도/Josa': True,
   '랑/Josa': False,
   '마치/Noun': False,
   '메리/Noun': False,
   '수업/Noun': False,
   '야/Josa': False,
   '이/Josa': False,
   '이쁘다/Adjective': False,
   '좋다/Adjective': True,
   '지루하다/Adjective': False},
  'pos'),
 ({'가/Josa': False,
   '고/Josa': False,
   '고양이/Noun': False,
   '난/Noun': True,
   '놀다/Verb': False,
   '는/Josa': False,
   '도/Josa': False,
   '랑/Josa': False,
   '마치/Noun': False,
   '메리/Noun': False,
   '수업/Noun': True,
   '야/Josa': False,
   '이/Josa': True,
   '이쁘다/Adjective': False,


변수명이 위에서는 all_words였다가 이번에는 tokens로, t에서 train_xy로 바뀌었을 뿐 원리는 똑같습니다

In [24]:
classifier = nltk.NaiveBayesClassifier.train(train_xy)
classifier.show_most_informative_features()

Most Informative Features
                  난/Noun = True              neg : pos    =      2.5 : 1.0
                 메리/Noun = False             neg : pos    =      2.5 : 1.0
            좋다/Adjective = False             neg : pos    =      1.5 : 1.0
                고양이/Noun = False             neg : pos    =      1.5 : 1.0
                  야/Josa = False             neg : pos    =      1.1 : 1.0
                 놀다/Verb = False             neg : pos    =      1.1 : 1.0
                  랑/Josa = False             neg : pos    =      1.1 : 1.0
                  도/Josa = False             neg : pos    =      1.1 : 1.0
                  고/Josa = False             neg : pos    =      1.1 : 1.0
                 마치/Noun = False             neg : pos    =      1.1 : 1.0


자 이제 검사의 조건들이 갖춰졌으니 다시 test_sentence를 가지고 분석을 해봅시다

In [25]:
test_sentence = [('난 수업이 마치면 코사다마랑 놀거야')]

# 똑같이 형태소 분석 
test_docs = pos_tagger.pos(test_sentence[0])
test_docs

[('난', 'Noun'),
 ('수업', 'Noun'),
 ('이', 'Josa'),
 ('마치', 'Noun'),
 ('면', 'Josa'),
 ('코', 'Noun'),
 ('사', 'Modifier'),
 ('다마', 'Noun'),
 ('랑', 'Josa'),
 ('놀거야', 'Verb')]

In [26]:
test_sent_features = {word: (word in tokens) for word in test_docs}
test_sent_features

{('난', 'Noun'): False,
 ('놀거야', 'Verb'): False,
 ('다마', 'Noun'): False,
 ('랑', 'Josa'): False,
 ('마치', 'Noun'): False,
 ('면', 'Josa'): False,
 ('사', 'Modifier'): False,
 ('수업', 'Noun'): False,
 ('이', 'Josa'): False,
 ('코', 'Noun'): False}

In [27]:
classifier.classify(test_sent_features)

'pos'

와우 🙊 이젠 긍정으로 잘 분석되었네요

## 문장의 유사도 측정하기  
많은 문장 혹은 문서들 중에서 유사한 문장을 찾아내는 방법입니다. 어떤 문장을 벡터로 표현할 수 있으면 벡터 간 거리를 구하는 방법으로 문장 간 유사도를 측정할 수 있습니다. 이해가 잘 안가셔도 괜찮습니다. 그냥 이런게 있구나~ 정도로 넘어가시면 됩니당 (사실 저도 정확한 원리까지는 잘 몰라요~ 2학기 머신러닝 커리에서 같이 공부해봅시다 👹)

In [28]:
# 텍스트의 특징을 추출하는 모듈
from sklearn.feature_extraction.text import CountVectorizer

# min_df: 특정 단어가 min_df보다 적은 빈도수가지면 그 단어 피처 제외 
vectorizer = CountVectorizer(min_df = 1)

In [29]:
# 유사한 문장을 연습용으로 사용해보기
# feat. 맞춤법 어떻하죠? 
contents = ['메리랑 놀러가고 싶지만 바쁜데 어떻하죠?',
            '메리는 공원에서 산책하고 노는 것을 싫어해요',
            '메리는 공원에서 노는 것도 싫어해요. 이상해요.',
            '먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요']

In [30]:
X = vectorizer.fit_transform(contents)
vectorizer.get_feature_names()

['것도',
 '것을',
 '곳으로',
 '공원에서',
 '그러질',
 '너무',
 '노는',
 '놀러가고',
 '떠나고',
 '메리는',
 '메리랑',
 '못하고',
 '바빠서',
 '바쁜데',
 '산책하고',
 '싫어해요',
 '싶은데',
 '싶지만',
 '어떻하죠',
 '여행을',
 '이상해요',
 '있어요']

어떻게 벡터화되는지는 교재 316쪽의 그림 8-19를 꼭 보고 이해해보기! 이렇게 벡터화하면 '메리는'과 '메리랑'을 다르게 생각되므로 더 한글 문장에 맞춰서 진행해봅시다.

In [31]:
from konlpy.tag import Okt 
okt = Okt()
contents_tokens = [okt.morphs(row) for row in contents]
contents_tokens

[['메리', '랑', '놀러', '가고', '싶지만', '바쁜데', '어떻하죠', '?'],
 ['메리', '는', '공원', '에서', '산책', '하고', '노', '는', '것', '을', '싫어해요'],
 ['메리', '는', '공원', '에서', '노', '는', '것', '도', '싫어해요', '.', '이상해요', '.'],
 ['먼',
  '곳',
  '으로',
  '여행',
  '을',
  '떠나고',
  '싶은데',
  '너무',
  '바빠서',
  '그러질',
  '못',
  '하고',
  '있어요']]

이제 자연어 처리 자체는 익숙하죵? '메리는'과 '메리랑'이 '메리','는','메리','랑'으로 잘 나눠집니다. 형태소 분석한 것을 하나의 문장으로 담을거에요. 대신, 각각의 형태소별로 띄어쓰기를 하여 sciket learn의 vectorizer 함수에서 사용하기 편하게 편집할겁니다

In [32]:
contents_for_vectorize = []

for content in contents_tokens:
  sentence =''
  for word in content: 
    sentence = sentence + ' ' + word 

  contents_for_vectorize.append(sentence)

contents_for_vectorize

[' 메리 랑 놀러 가고 싶지만 바쁜데 어떻하죠 ?',
 ' 메리 는 공원 에서 산책 하고 노 는 것 을 싫어해요',
 ' 메리 는 공원 에서 노 는 것 도 싫어해요 . 이상해요 .',
 ' 먼 곳 으로 여행 을 떠나고 싶은데 너무 바빠서 그러질 못 하고 있어요']

In [33]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape 
num_samples, num_features

(4, 20)

In [34]:
vectorizer.get_feature_names()

['가고',
 '공원',
 '그러질',
 '너무',
 '놀러',
 '떠나고',
 '메리',
 '바빠서',
 '바쁜데',
 '산책',
 '싫어해요',
 '싶은데',
 '싶지만',
 '어떻하죠',
 '에서',
 '여행',
 '으로',
 '이상해요',
 '있어요',
 '하고']

말뭉치가 잘 형성되었네요! 1개 이하로 나오는 것은 말뭉치에 왜 넣지 않은지는 앞서 객체 생성할 때 min_df 파라미터를 1로 설정해주었기 때문이에요. 이 리스트를 numpy의 array 형식으로 바꾸어주고, 행과 열을 바꿔서 표시해주려고 합니다. 결과는 318쪽 그림 8-21에 있어요.

In [35]:
X.toarray().transpose()

array([[1, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 1, 1, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 1]])

transpose는 행과 열을 바꾼 행렬인 전치행렬을 만들라는 함수인데요. 이게 알고보니까 수학 개념이더라고요. 확실히 수학을 알면 코딩도 조금은 쉬워지는 것 같습니다

여튼! 이렇게 학습 데이터를 기반으로 말뭉치를 잘 만들어주었으니, 테스트 데이터를 넣어봐야겠지요. 새로운 문장을 동일하게 벡터화해서 각 벡터들 사이의 거리를 구하면 문장 간의 유사도를 측정할 수 있다는 개념이에요.

In [36]:
new_post = ['메리랑 공원에서 산책하고 놀고 싶어요']
new_post_tokens = [okt.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
  sentence =''
  for word in content: 
    sentence = sentence + ' ' + word

  new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[' 메리 랑 공원 에서 산책 하고 놀고 싶어요']

벡터화하기 쉽도록 문장을 잘 가공해주었고, 벡터화를 시켜주면 됩니다.

In [37]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

## 유클리드 거리 유사도
자! 그럼 이 상태에서 어떻게 유사도 측정을 하느냐? 크게 두가지 방법이 있는 것 같습니다. 하나는 거리 차이를 이용하는 방법, 다른 하나는 각도 차이를 이용하는 방법이 있어요. 여기에서는 거리 차이를 이용해서 측정하는 방법을 사용하는데요, 정확히는 유클리드 거리 유사도를 사용하여 문장 간의 유사도를 측정해보는 방법입니다. 거리를 기반으로 하면, 특정 좌표 주변과 가까이에 있는 것이 유사도가 높다고 판단하는 것이죠.  

[이미지예시](https://d1z1a3q7uy2q8k.cloudfront.net/blog_images/image_5394346701474615621316.png, https://specialscene.tistory.com/48)


벡터 a의 길이는 놈(norm)으로 정의하는데요, 이를 구현해주는 코드가 np.linalg.norm()이라고 러프하게 이해하시면 될 듯 합니다..ㅎㅎ 벡터에 대해서 알아보고 싶으면 slack 채널에 mathematics 방으로 오세요. 같이 선형대수를 배워봅시다!

[참고: 벡터의 길이](https://datascienceschool.net/view-notebook/dd1680bfbaab414a8d54dc978c6e883a/)

In [38]:
import scipy as sp 

def dist_raw(v1, v2):
  delta = v1 - v2 
  return sp.linalg.norm(delta.toarray())

In [39]:
best_doc = None
# 65535가 무슨 의미인지는 모르겠네요... 
best_dist = 65535 
best_i = None 

for i in range(0, num_samples):
  post_vec = X.getrow(i)
  d = dist_raw(post_vec, new_post_vec)

  # i는 문장 번호, d는 유사도
  print('== Post %i with dist=%.2f    : %s' % (i,d,contents[i]))

# best_dist가 계속 낮게 수정되는 것! (낮을수록 유사도 높음)
  if d < best_dist: 
    best_dist = d
    best_i = i

== Post 0 with dist=3.00    : 메리랑 놀러가고 싶지만 바쁜데 어떻하죠?
== Post 1 with dist=1.00    : 메리는 공원에서 산책하고 노는 것을 싫어해요
== Post 2 with dist=2.00    : 메리는 공원에서 노는 것도 싫어해요. 이상해요.
== Post 3 with dist=3.46    : 먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요


In [40]:
print('Best post is %i, dist = %.2f' % (best_i, best_dist))
print('-->', new_post)
print('---->', contents[best_i])

Best post is 1, dist = 1.00
--> ['메리랑 공원에서 산책하고 놀고 싶어요']
----> 메리는 공원에서 산책하고 노는 것을 싫어해요


가장 유사도가 높은 문장이 바로 '메리는 공원에서 산책하고 노는 것을 싫어해요'라는 문장이네요! 의미는 반대나 조합이 비슷해서 유사도를 높게 측정한 듯 합니다.

In [41]:
# contents와 새로운 문장이 벡터화된 결과 

for i in range(0, len(contents)):
  # 벡터화한 것의 row만 가져와봐라
  print(X.getrow(i).toarray())

print('------------------')
print(new_post_vec.toarray())

[[1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0]]
[[0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1]]
[[0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0]]
[[0 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 0 1 1]]
------------------
[[0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1]]


In [42]:
def dist_norm(v1, v2): 
  v1_normalized = v1 / sp.linalg.norm(v1.toarray())
  v2_normalized = v2 / sp.linalg.norm(v2.toarray())

  delta = v1_normalized - v2_normalized 

  return sp.linalg.norm(delta.toarray())

In [43]:
best_doc = None 
best_dist = 65535
best_i = None 

for i in range(0, num_samples): 
  # 앞에서 array로 바꿔줬으니 다시 array로 바꿔줄필요 없음 
  post_vec = X.getrow(i)
  d = dist_norm(post_vec, new_post_vec)

  print("== Post %i with dist=%2f   : %s" % (i,d,contents[i]))

  if d < best_dist: 
    best_dist = d
    best_i = i

== Post 0 with dist=1.278613   : 메리랑 놀러가고 싶지만 바쁜데 어떻하죠?
== Post 1 with dist=0.417442   : 메리는 공원에서 산책하고 노는 것을 싫어해요
== Post 2 with dist=0.894427   : 메리는 공원에서 노는 것도 싫어해요. 이상해요.
== Post 3 with dist=1.304553   : 먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요


In [44]:
print('Best post is %i, dist = %.2f' % (best_i, best_dist))
print('-->', new_post)
print('---->', contents[best_i])

Best post is 1, dist = 0.42
--> ['메리랑 공원에서 산책하고 놀고 싶어요']
----> 메리는 공원에서 산책하고 노는 것을 싫어해요


## TF-IDF  
tf(term frequency)와 idf(inverse document frequency)는 텍스트 마이닝에서 사용하는 일종의 단어별로 부과하는 가중치입니다. tf는 어떤 단어가 문서 내에서 자주 등장할수록 중요도가 높을 것으로 보는 것이고, idf는 비교하는 모든 문서에 만약 같은 단어가 있다면 이 단어는 핵심 어휘일지는 모르지만 문서 간의 비교에서는 중요한 단어가 아니라는 뜻으로 보는 것이에요. 교재 322쪽에서는 직접 함수를 만들어 예제에 적용해본 결과가 있지만, (이해가 중요할 수도 있겠지만) 이것의 의미를 알고 일단 우리는 적용해서 해석할 수 있기만 하면 돼요! 우리가 사용할 것은 **TF-IDF** 기반의 벡터화이고, 개별 문서에서 자주 나타나는 단어에 높은 가중치를 주되, 모든 문서에서 전반적으로 자주 나타나는 단어에 대해서는 페널티를 주는 방식으로 값을 부여한다고 보면 됩니다

정독해보기 [TF-IDF의 개념](https://charsyam.wordpress.com/2017/04/08/%EC%9A%A9%EC%96%B4-%EC%A0%95%EB%A6%AC-%EC%9E%85-%EA%B0%9C%EB%B0%9C%EC%9E%90%EB%A5%BC-%EC%9C%84%ED%95%9C-tf-idf/)

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer 
# decode_error은 디코딩 에러 무시한다는 말! 
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

contents_tokens = [okt.morphs(row) for row in contents]

contents_for_vectorize = []

for content in contents_tokens: 
  sentence = ''
  for word in content: 
    sentence = sentence + ' ' + word 

  contents_for_vectorize.append(sentence)

X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape 
num_samples, num_features

(4, 20)

In [46]:
vectorizer.get_feature_names()

['가고',
 '공원',
 '그러질',
 '너무',
 '놀러',
 '떠나고',
 '메리',
 '바빠서',
 '바쁜데',
 '산책',
 '싫어해요',
 '싶은데',
 '싶지만',
 '어떻하죠',
 '에서',
 '여행',
 '으로',
 '이상해요',
 '있어요',
 '하고']

말뭉치들이 잘 생성되었네요. 그러면 새로운 문장으로 다시 테스트 해봅시다.

In [47]:
new_post = ['근처 공원에 메리랑 놀러가고 싶네요.']
new_post_tokens = [okt.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens: 
  sentence = ''
  for word in content: 
    sentence = sentence + ' ' + word 

  new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[' 근처 공원 에 메리 랑 놀러 가고 싶네요 .']

형태소 분석이 잘 되었구요, 이를 벡터화하여 새롭게 유사도를 측정해봅시다.

In [48]:
best_doc = None 
best_dist = 65535
best_i = None 

for i in range(0, num_samples): 
  post_vec = X.getrow(i)
  d = dist_norm(post_vec, new_post_vec)

  print("== post %i with dist=%.2f    : %s" % (i,d,contents[i]))

  if d < best_dist: 
    best_dist = d
    best_i = i 

print("Best post is %i, dist = %.2f" % (best_i, best_dist))
print('-->', new_post)
print('---->', contents[best_i])

== post 0 with dist=1.32    : 메리랑 놀러가고 싶지만 바쁜데 어떻하죠?
== post 1 with dist=0.43    : 메리는 공원에서 산책하고 노는 것을 싫어해요
== post 2 with dist=0.95    : 메리는 공원에서 노는 것도 싫어해요. 이상해요.
== post 3 with dist=1.33    : 먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요
Best post is 1, dist = 0.43
--> ['근처 공원에 메리랑 놀러가고 싶네요.']
----> 메리는 공원에서 산책하고 노는 것을 싫어해요


이런 방식으로 벡터화를 해도 마찬가지로 '메리는 공원에서 산책하고 노는 것을 싫어해요'가 가장 높은 유사성을 보이네요^^

휴... 머신러닝 맛보기 커리는 여기서 끝입니다. 이번 주차에는 공부할게 정말 많네요! 그래도 이게 다 피가되고 살이되는 그런겁니다~ ㅎㅎㅎ 지금은 이해못하더라도 아 이런 기법이 있구나 정도만 알아두시면 이번 교안은 잘 공부하신겁니다.

다음 커리에서는 자연어처리를 실제로 어떻게 활용하는 법을 청와대 청원 크롤링 & 분석 프로젝트를 통해 익힐겁니다. 그럼 다음 커리에서 또 봐요~👨‍💻